<center> <b> Anti-Fraud Ops Request | 30NOV23 </center>

Hi @Gaurav and Team,
 
 
Good afternoon.
 
Can you please help us extract the PIDs related to the JIRA tickets and TSAs in the attached file (Sheet2 and Sheet3)?

We need the data for an upcoming IT development related to fraud account tagging. Thank you.

Ler me know if you have questions.


Regards,
Mela


In [1]:
# Import Libraries

import pandas as pd  
import numpy as np  


from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

## Read the data from Sheet3

In [2]:
dfinputs3 = pd.read_excel(r"D:\MyStuff\Mela Sombillo csombillo1@tonikbank.com\Anti-Fraud Ops Request  30NOV23\Fraud PIDs v1.xlsx", sheet_name='Sheet3')
dfinputs3.shape

(328, 3)

## Check the data in the sheet

In [3]:
dfinputs3.head()

,Fraud Type,JIRA,PID
0,KYC Suspicious,CC-1004163,NaN
1,KYC Suspicious,CC-1005069,NaN
2,KYC Suspicious,CC-1090117,NaN
3,KYC Suspicious,CC-1090117,NaN
4,KYC Suspicious,CC-1231501,NaN


## Run query to find relevant data from cc_ticket table

In [4]:
sq = """select * from prj-prod-dataplatform.jira_raw.cc_tickets limit 100;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID c0ac68df-33de-40b7-85ed-18d9b9c79990 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


## Check the output of previous query from jira ticket table

In [5]:
pd.set_option('display.max_columns', None)
df.head()

,issue_id,issue_key,parent_id,parent_issue_key,status,summary,issue_type,project_key,project_name,project_type,priority,assignee,reporter,created_date,updated_date,last_viewed,components,fix_version,due_date,labels,description,story_points,resolution_date,resolution_description,originalEstimate,remainingEstimate,timeSpent,originalEstimateSeconds,remainingEstimateSeconds,timeSpentSeconds,work_ratio,sprint_id,sprint_name,sprint_state,sprint_goal,sprint_start_date,sprint_end_date,squad_name,department_unit_name,workgroup_name,support_team_name,dev_status,module_name,detailed_root_cause,change_date,changed_items_json,resource_name,issue_links,affects_version,assignee_account_id,reporter_account_id,change_author_name,change_author_account_id,status_category,worklog_struc,ticket_creation_reason,bug_resolution
0,861392,CC-633180,None,None,Closed,Inquiries | General Inquiries,Inquiry,CC,CC-CASE,service_desk,4,CC-Supervisor-Group,Aaron Tordil,2022-07-10 15:18:59,2022-07-10 15:28:44,None,None,None,None,None,"The issue is considered finished, the resoluti...",None,2022-07-10T15:19:50.170+0800,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,LEVEL-1,None,None,None,"[2022-07-10T15:28:44.508+0800, 2022-07-10T15:1...","[[{""field"":""status"",""fieldId"":""status"",""fieldt...",,[],None,None,None,[],[],None,None,None,None
1,2870964,CC-2099138,None,None,Closed,Inquiries | General Inquiries,Inquiry,CC,CC-CASE,service_desk,4,CC-Supervisor-Group,Aaron Tordil,2023-10-13 13:50:52,2023-10-13 13:58:42,None,None,None,None,None,"The issue is considered finished, the resoluti...",None,2023-10-13T13:54:58.007+0800,Done,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,LEVEL-1,None,None,None,"[2023-10-13T13:58:42.811+0800, 2023-10-13T13:5...","[[{""field"":""status"",""fieldId"":""status"",""fieldt...",,[],None,5f2251d9ef11df00259b53f3,61930bf63618cd006f0e39c2,"[Grace Ann Basa Salas, Aaron Tordil, Aaron Tor...","[61bc01bd89e89900710644ee, 61930bf63618cd006f0...",Done,[],None,None
2,385613,CC-283721,None,None,Closed,Inquiries | General Inquiries,Inquiry,CC,CC-CASE,service_desk,4,CC-Supervisor-Group,Aaron Tordil,2022-02-01 07:08:28,2022-02-01 15:32:45,None,None,None,None,None,"The issue is considered finished, the resoluti...",None,2022-02-01T07:09:27.452+0800,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,LEVEL-1,None,None,None,"[2022-02-01T15:32:45.449+0800, 2022-02-01T07:0...","[[{""field"":""status"",""fieldId"":""status"",""fieldt...",,[],None,None,None,[],[],None,None,None,None
3,1088030,CC-807168,None,None,Closed,Inquiries | General Inquiries,Inquiry,CC,CC-CASE,service_desk,4,CC-Supervisor-Group,Aaron Tordil,2022-08-23 17:03:55,2022-08-23 17:45:32,None,None,None,None,None,"The issue is considered finished, the resoluti...",None,2022-08-23T17:04:22.658+0800,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,LEVEL-1,None,None,None,"[2022-08-23T17:45:32.501+0800, 2022-08-23T17:0...","[[{""field"":""status"",""fieldId"":""status"",""fieldt...",,[],None,None,None,[],[],None,None,None,None
4,381384,CC-280898,None,None,Closed,Inquiries | General Inquiries,Inquiry,CC,CC-CASE,service_desk,4,CC-Supervisor-Group,Aaron Tordil,2022-01-29 21:06:22,2022-01-29 21:19:43,None,None,None,None,None,"The issue is considered finished, the resoluti...",None,2022-01-29T21:07:28.792+0800,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,LEVEL-1,None,None,None,"[2022-01-29T21:19:43.706+0800, 2022-01-29T21:0...","[[{""field"":""status"",""fieldId"":""status"",""fieldt...",,[],None,None,None,[],[],None,None,None,None


In [6]:
df['summary'].unique()

array([' Inquiries | General Inquiries', 'Inquiry | Account Related'],
      dtype=object)

## Input the Jira column values as tuple so that we can use it in the query

In [7]:
a = tuple(dfinputs3['JIRA'])


## Run the query to find the data from cc_tickets table for all JIRA values from sheet3

In [8]:
sq = f"""select issue_key, summary from prj-prod-dataplatform.jira_raw.cc_tickets where issue_key in {a};"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df.head() 

Job ID aeb192b6-549c-45d3-9124-6318f3a81d5e successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,issue_key,summary
0,CC-468920,KYC Fraud Suspicious | 1198697
1,CC-430206,KYC Fraud Suspicious | 1349127
2,CC-430310,KYC Fraud Suspicious | 1349165
3,CC-458931,KYC Fraud Suspicious | 1366034
4,CC-248133,Loan | Suspicious Transaction | 1193909 | FAIT...


## Since in the summary column we have text as well as the customer id, we have to use regex to split the value to a new column match

In [9]:
import re

# Use regular expression to extract only the number
df['match'] = df['summary'].str.extract(r'\b(\d+)\b')
df.head()

,issue_key,summary,match
0,CC-468920,KYC Fraud Suspicious | 1198697,1198697
1,CC-430206,KYC Fraud Suspicious | 1349127,1349127
2,CC-430310,KYC Fraud Suspicious | 1349165,1349165
3,CC-458931,KYC Fraud Suspicious | 1366034,1366034
4,CC-248133,Loan | Suspicious Transaction | 1193909 | FAIT...,1193909


## Merge both the spreadsheet dataframe and cc ticket query dataframe using the common column JIRA and Issue_key

In [10]:
df2 = dfinputs3.merge(df, left_on='JIRA', right_on='issue_key', how = 'left')
df2.head()

,Fraud Type,JIRA,PID,issue_key,summary,match
0,KYC Suspicious,CC-1004163,NaN,CC-1004163,KYC Fraud Suspicious | 1688457,1688457
1,KYC Suspicious,CC-1005069,NaN,CC-1005069,KYC Fraud Suspicious | 1694891,1694891
2,KYC Suspicious,CC-1090117,NaN,CC-1090117,KYC Fraud Suspicious | 1750896,1750896
3,KYC Suspicious,CC-1090117,NaN,CC-1090117,KYC Fraud Suspicious | 1750896,1750896
4,KYC Suspicious,CC-1231501,NaN,CC-1231501,KYC Fraud Suspicious | 1824238,1824238


## Check the null value. Only 1 customer id details are missing

In [11]:
df2.isnull().sum()

Fraud Type      0
JIRA            0
PID           328
issue_key       0
summary         0
match           1
dtype: int64

## find out the exact row with the missing value

In [12]:
df2[df2.match.isnull()]

,Fraud Type,JIRA,PID,issue_key,summary,match
287,Loans Auto Reject,CC-1667034,NaN,CC-1667034,Lending (TUL) - Potential fraud attempt,NaN


We can see that there is no mention of customer id in the summary as it was with other rows

## Drop the PID and issue key columns from the newly formed dataframe

In [13]:
df2.drop(columns=['PID', 'issue_key'], inplace=True)
df2.head()

,Fraud Type,JIRA,summary,match
0,KYC Suspicious,CC-1004163,KYC Fraud Suspicious | 1688457,1688457
1,KYC Suspicious,CC-1005069,KYC Fraud Suspicious | 1694891,1694891
2,KYC Suspicious,CC-1090117,KYC Fraud Suspicious | 1750896,1750896
3,KYC Suspicious,CC-1090117,KYC Fraud Suspicious | 1750896,1750896
4,KYC Suspicious,CC-1231501,KYC Fraud Suspicious | 1824238,1824238


## Rename column match as PID 

In [14]:
df2.rename(columns={'match':'PID'}, inplace=True)
df2.columns

Index(['Fraud Type', 'JIRA', 'summary', 'PID'], dtype='object')

## Reshape the dataframe as per the order in spreadsheet

In [15]:
df2 = df2[['Fraud Type', 'JIRA', 'PID', 'summary']]

In [16]:
df2.head()

,Fraud Type,JIRA,PID,summary
0,KYC Suspicious,CC-1004163,1688457,KYC Fraud Suspicious | 1688457
1,KYC Suspicious,CC-1005069,1694891,KYC Fraud Suspicious | 1694891
2,KYC Suspicious,CC-1090117,1750896,KYC Fraud Suspicious | 1750896
3,KYC Suspicious,CC-1090117,1750896,KYC Fraud Suspicious | 1750896
4,KYC Suspicious,CC-1231501,1824238,KYC Fraud Suspicious | 1824238


## Save the data to an excel sheet

In [17]:
df2.to_excel("Fraud_PID_v1_Sheet3data_20231205.xlsx", index = False, sheet_name='Data')